# Overview of ideas

There are several topics and problems here.

## Virtual Patients

We are solving this problem by applying DP to GP. We take the distribution of data and calculate the GP mean function (for example) then perterb it using a sample from the same GP's kernel.

We will need to use inducing inputs for two reasons:

1. It will allow the points to be spaced sufficiently far apart to ensure that bounds we chose for the noise remain valid.
2. It means that the sensitivity of the movement of individual data will be limited. For example if a point is moved across the whole input domain, the effect on the inducing inputs will be small.

The next step is to bound this change. We need to do this in two ways:

1. How much can the inducing inputs move in the 'most common cases'
2. How likely is it that this is not one of the 'most common cases' (this allows us to use the $\delta$ parameter within the DP definition).
    
## User-centric data and a distributed learning platform

We want to keep user data on the user device (or on a cloud service that only they have the decryption key to).

We want to train a GP on anonymous data from many users.

We might want to do further training locally?

### Ways to anonymise?

#### Differential Privacy

use the local model: Add enough noise so that individual rows can be released

#### Differential Privacy with Multiparty computation

DP benefits from aggregation (less noise is required, e.g. for the mean of a variable). Multiparty computation requires a large number of messages to be passed, that increases quickly with the number of parties. Maybe a compromise can be found between the two (group people into subgroups, each of which has MPC used within it to produce a noisy output, then these outputs are combined).

Example:

1 million people want to find the mean of their incomes. Let's assume they vary between 0 and £100,000.

MPC could anonymously find this value (and then add noise proportional to 100,000/1,000,000 = 0.1).

This will require 1 million message passes.

Alternatively:

We could split the million people into 10,000 groups. Each will message pass 100 times. The noise added to each mean estimate will be 100,000/100 = 1000. We have 10,000 groups, so our estimate error will be 1000/sqrt(10,000) = 1000/100 = 10. So 100 times more noise, but it would take 10,000 less time.

I don't know anything about MPC but it seems that usually the message count scales quickly with N - not just O(N). I'll look for a more concrete example.

I don't know if MPC is ever practial. It requires communication between users... or their data stores.

#### Differential Privacy over Parameters

This is related to the work from Temo.

If multiple people are sent queries for the sum of subsets of parameters, the space for the centralised machine learner will consist of lots of planes passing at 45 degrees from sets of axes. A zero-mean prior will help resolve these. With enough of them could the learner make inference about the whole dataset?

## How much computation can happen locally?

To reason about this problem I feel an example is useful.

Consider if we speak to myfitnesspal and suggest a button be added to allow migraine suffers to report when they have a migraine. One could then figure out what foods are triggers.

    Migraine:  |           |  | |             |
    Foods:   a b c  a    b  c  b b  b b a   b a  
    
    Migraine:     | |     |     |     |    |   
    Foods:     c c  c d  ca     a a  a   b
    
    Migraine:     |                    | |      
    Foods:    c d    b                 d b  a 
    
    Migraine:                    |     || |   
    Foods:    c   d    c   d   a  b     a b  c
     
A tempting start might be a GLM or (obviously) a GP.

Maybe we would start by looking for likely links between foods and migraines. Some people who are affected by food 'a' are also affected by food 'c'? Each of these could be done locally. In the case of the GLM a weight matrix would be output, the weights could be protected using standard Differential Privacy (local model). At the server the weight matrices could be combined, e.g. finding which weights are often correlated (can this be done if we've added DP local-model noise?). This info can be returned to the users, who can use the refined weight matrices to help increase the guess of their own triggers.

How would this be done with GPs? We're interested in the weight of each foot item...
    
    
# To Do

 - Read http://papers.nips.cc/paper/5392-extremal-mechanisms-for-local-differential-privacy